## Utils

In [2]:
import telnetlib 
import json

def readline():
    return tn.read_until(b"\n")

def json_recv():
    line = readline()
    return json.loads(line.decode())

def json_send(hsh):
    request = json.dumps(hsh).encode()
    tn.write(request)

HOST = 'socket.cryptohack.org' 

/var/folders/s6/v17rrjkx205gt68br6qh7zqr0000gn/T/ipykernel_1504/3786777262.py:1: DeprecationWarning: 'telnetlib' is deprecated and slated for removal in Python 3.13
  import telnetlib


# ONE TIME PAD

### Gotta Go Fast 

Since server uses current time as input to SHA256 as key we can send 2 challenges fast and get same key for encryption of flag and our message

In [13]:
from pwn import xor 
PORT = 13372
tn = telnetlib.Telnet(HOST, PORT)

readline()

m_prime = b'crypto{????????????????????}'
forgery = {}
forgery['option'] = 'encrypt_data'
forgery['input_data'] = m_prime.hex() 

json_send(forgery)
r = json_recv()
c_prime = bytes.fromhex(r['encrypted_data']) 
k= xor(c_prime, m_prime)

json_send({'option':'get_flag'})
r = json_recv()
c = bytes.fromhex(r['encrypted_flag'])
print(xor(c, k).decode())

crypto{t00_f4st_t00_furi0u5}


### No leaks

So it checks if ciphertext has same byte as flag, so basically if we get ciphertext back, we can remove from guesses all the characters we've seen there for our bytes. 

In [36]:
import base64 

PORT = 13370
tn = telnetlib.Telnet(HOST, PORT)

readline()
FLAG = "crypto{????????????}"

guesses = [] 
for _ in range(len(FLAG)-8): 
    guesses.append([i for i in range(33,127)])

while True: 
    progress = sum([len(i) for i in guesses]) 
    
    if progress == 12: 
        break 

    json_send({'msg': 'request'})
    r = json_recv()
    
    if "error" not in r:  
        c = base64.b64decode(r['ciphertext'])
        c = c[7:len(FLAG)-1]
        for i in range(len(c)): 
            try: 
                guesses[i].remove(c[i]) 
            except Exception: 
                continue 
f = "crypto{"           
for i, g in enumerate(guesses): 
    f+=chr(g[0]) 
print(f+"}")

crypto{unr4nd0m_07p}


# ELGAMAL

### Bit by Bit

$Padding = 256^t, t\in [2..q] = 2^{8t}$

```py
me = padding << 1 + m % 2
```

$m_{padded} = \begin{cases}
  2^{8t+1} & \text{if } m \equiv 0 (\bmod2) \\
  2^{8t+2} & \text{if } m \equiv 1 (\bmod2)
\end{cases}$

We can write this term as $2^{2(4t+1)}$, so we use Legrende's symbol to check if our $m_{padded}$ is quadratic residue $\bmod q$, i.e if there exists some $x$, s.t.

$x^2 \equiv m_{padded} \bmod q$ because of term 2 we expressed. 

Now we also have to check whether $g$ is quadratic residue mod q

In [43]:
from sympy.ntheory import legendre_symbol
from Crypto.Util.number import long_to_bytes

q = 117477667918738952579183719876352811442282667176975299658506388983916794266542270944999203435163206062215810775822922421123910464455461286519153688505926472313006014806485076205663018026742480181999336912300022514436004673587192018846621666145334296696433207116469994110066128730623149834083870252895489152123
g = 104831378861792918406603185872102963672377675787070244288476520132867186367073243128721932355048896327567834691503031058630891431160772435946803430038048387919820523845278192892527138537973452950296897433212693740878617106403233353998322359462259883977147097970627584785653515124418036488904398507208057206926

assert(legendre_symbol(g,q) == 1)

data = None 

def to_tuple(str):
    str = str.strip()
    str = str[1:-1]
    str = str.split(",")
    return str[0],str[1]

with open("data/bit_by_bit.txt") as f: 
    data = f.readlines()

FLAG = ""
for i in range(1,len(data),2): 
    c2 = int(to_tuple(data[i])[1].split("=")[1],16)
    if legendre_symbol(c2,q) ==  1:
        FLAG+="1"
    else:
        FLAG+="0"

FLAG = FLAG[::-1] 
print(long_to_bytes(int(FLAG,2)).decode())

crypto{s0m3_th1ng5_4r3_pr3served_4ft3r_encrypti0n}


# SECRET SHARING SCHEMES

### Armory

https://en.wikipedia.org/wiki/Shamir%27s_secret_sharing

We've got the evaluation of polynomial $y$ and $a_1$ given to us, since polynomial is of degree 2 and following the code we derive that $x=a1$ 
and that $a_2=H(a_1)$ $H$ - SHA256. 

$y = a_2x^2 + a_1x + f \bmod p$
 
$f = y - a_2a_1^2 - a_1^2 \bmod p$ 


In [9]:
from Crypto.Util.number import long_to_bytes, bytes_to_long
import hashlib  
a1,y = (105622578433921694608307153620094961853014843078655463551374559727541051964080, 
 25953768581962402292961757951905849014581503184926092726593265745485300657424)

p = 77793805322526801978326005188088213205424384389488111175220421173086192558047
a2 = bytes_to_long(hashlib.sha256(long_to_bytes(a1)).digest()) 

print(long_to_bytes((y-a2*a1*a1 - a1*a1)%p).decode())

crypto{fr46m3n73d_b4ckup_vuln?}


### Toshi's Treasure

https://en.wikipedia.org/wiki/Shamir%27s_secret_sharing


$f(0) = a_0 = \sum y_j\Pi\frac{x_m}{x_m-x_j}$

We send $y = 0$ so we get everything but first 

We calculate hyper's $y$ and send it back

We combine with our y and the product with everything but first

In [70]:
from python import utils

my_1k_wallet_privkey = int("8b09cfc4696b91a1cc43372ac66ca36556a41499b495f28cc7ab193e32eadd30",16)
p = 2**521 - 1 # 13th Mersenne prime 


c = utils.Challenge(int(13384))


r = c.json_recv()
print(r)
other_parties = [2, 3, 4, 5]
me = r['x']
my_y = r['y']

print(c.json_recv())
print(c.json_recv())
print(c.json_recv())
print(c.json_recv())

to_send = {}
to_send['sender'] = 'kriska'
to_send['msg'] = 'sending'
to_send['x'] = me 
to_send['y'] = hex(0)

c.json_send(to_send)

priv_key_fail = int(c.json_recv()['privkey'],16) 
print(c.json_recv())
print(c.json_recv())


pi_x = 1
for x in other_parties:
    pi_x *= (x * pow((x - me), -1, p))
    pi_x%=p

y = ((my_1k_wallet_privkey - priv_key_fail)*pow(pi_x,-1,p))%p

to_send['y'] = hex(y)
c.json_send(to_send)

print(c.json_recv())
print(c.json_recv())
print(c.json_recv())

real_priv_key = (priv_key_fail + int(my_y, 16)*pi_x) % p
to_send = {'privkey': hex(real_priv_key)}

c.json_send(to_send)
print(c.json_recv())

{'sender': 'your_share', 'x': 6, 'y': '0x1c13501adb88e6e8f1cc36d929704de302e0277c3e7a2cc591c6e2bf9b1641c98b4a7255e4e7e1fedb87825f02f3e6e434b028a1497bcfe027c1104013f7af3dd37'}
{'sender': 'mpeg', 'msg': "alright, let's go! I have x=2 and I'll start by secretly submitting my y point to our key-combiner", 'x': 2, 'y': '0x???'}
{'sender': 'codecrafting', 'msg': 'submitting my point', 'x': 3, 'y': '0x???'}
{'sender': 'RichGirlOnLSD', 'msg': 'sending in my x=4 point, so excited!', 'x': 4, 'y': '0x???'}
{'sender': 'pogo', 'msg': 'sent my x=5 val, your turn next hyper', 'x': 5, 'y': '0x???'}
{'sender': 'mpeg', 'msg': 'WTF, this privkey is invalid!'}
{'sender': 'codecrafting', 'msg': "hmm, hopefully someone just missed a character when pasting. let's try this one more time, and send in all our shares again. your turn hyper"}
{'sender': 'ssss-combine', 'privkey': '8b09cfc4696b91a1cc43372ac66ca36556a41499b495f28cc7ab193e32eadd30'}
{'sender': 'mpeg', 'msg': 'this privkey links to a wallet which onl